# Mini-Projet Intelligence Artificielle : Prédiction du Revenu Annuel d'un Marocain

**Membres du Groupe :**
*   Saad Barhrouj
*   Nassim El Kaddaoui
*   Youness Kihel

**Année :** 2ème année Cycle d’Ingénieurs – GI
**Module :** Intelligence Artificielle
**Encadrant :** Y. EL YOUNOUSSI
**Année Universitaire :** 2024-2025

## Objectif du Mini-Projet
L'objectif général de ce mini-projet est de construire un pipeline complet de Machine Learning en Python pour prédire le revenu annuel des Marocains à partir de données simulées réalistes. Le projet couvre toutes les étapes de développement d’un modèle de Machine Learning : compréhension des données, préparation des données, modélisation, évaluation et déploiement.


## 0. Importation des Librairies et Configuration Initiale

Cette section importe les librairies Python nécessaires pour l'analyse et la manipulation des données, ainsi que pour la visualisation. Des configurations pour l'affichage des DataFrames Pandas sont également définies pour une meilleure lisibilité.

In [13]:
# Librairies de base pour la manipulation de données
import pandas as pd
import numpy as np

# Librairies pour la visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Librairie pour le profilage de données (alternative à Sweetviz)
from ydata_profiling import ProfileReport # ou from pandas_profiling import ProfileReport pour les anciennes versions

# Options d'affichage pour Pandas
pd.set_option('display.max_columns', None) # Afficher toutes les colonnes
pd.set_option('display.max_rows', 100)      # Afficher jusqu'à 100 lignes
pd.set_option('display.float_format', lambda x: '%.2f' % x) # Formater les floats à 2 décimales

# Pour rendre les graphiques Matplotlib plus esthétiques (style seaborn)
plt.style.use('seaborn-v0_8-whitegrid') # Vous pouvez choisir un autre style seaborn

# Ignorer les avertissements (à utiliser avec précaution, notamment en phase d'exploration)
import warnings
warnings.filterwarnings('ignore')

# Constante pour la reproductibilité des résultats lors des étapes aléatoires (train_test_split, modèles, etc.)
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE) # Assure la reproductibilité pour les opérations numpy aléatoires du notebook (la génération du CSV est séparée)


## 1. Compréhension des Données

La première étape cruciale dans tout projet de Machine Learning est de bien comprendre les données avec lesquelles nous travaillons. Cela implique de charger le dataset, d'inspecter sa structure, ses types de données, ses statistiques descriptives, et d'explorer les relations entre les variables. Les données utilisées ici ont été générées par le script `generate_dataset-kihel.py`.

### 1.1. Chargement du Dataset

Nous commençons par charger le fichier CSV `dataset_revenu_marocains.csv` (préalablement généré) dans un DataFrame Pandas.

In [14]:
# Chemin vers le fichier dataset (s'assurer qu'il est accessible)
file_path = "dataset_revenu_marocains.csv" 

# Chargement du dataset
try:
    df_original = pd.read_csv(file_path) # On le nomme df_original pour garder une copie intacte
    df = df_original.copy() # On travaillera sur une copie pour éviter de recharger en cas d'erreur
    print(f"Dataset '{file_path}' chargé avec succès.")
    print(f"Le dataset contient {df.shape[0]} lignes et {df.shape[1]} colonnes.")
except FileNotFoundError:
    print(f"ERREUR : Le fichier '{file_path}' n'a pas été trouvé. Veuillez vérifier le chemin et le nom du fichier.")
    df = None # Pour éviter les erreurs dans les cellules suivantes si le fichier n'est pas trouvé
except Exception as e:
    print(f"Une erreur est survenue lors du chargement du fichier : {e}")
    df = None

Dataset 'dataset_revenu_marocains.csv' chargé avec succès.
Le dataset contient 40000 lignes et 18 colonnes.


### 1.2. Affichage des Données : Aperçu Initial

Pour avoir une première idée du contenu du dataset, nous affichons les 10 premières et les 5 dernières instances. Cela nous permet de voir les noms des colonnes et quelques exemples de valeurs.

In [15]:
if df is not None: 
    print("Affichage des 10 premières lignes du dataset :")
    display(df.head(10))

    print("\nAffichage des 5 dernières lignes du dataset :")
    display(df.tail(5))
else:
    print("Le DataFrame n'a pas été chargé. Impossible d'afficher les données.")

Affichage des 10 premières lignes du dataset :


,Age,Categorie_age,Sexe,Milieu,Region_geographique,Etat_matrimonial,Niveau_education,Annees_experience,CSP,Secteur_emploi,Propriete_immobiliere,Vehicule_motorise,Terrain_agricole,Revenu_secondaire,Revenu_Annuel,Revenu_Mensuel,Adresse_Email,CIN
0,43,Adulte,Homme,Rural,Nord,Célibataire,Secondaire,12.00,Employés,Privé,Oui,Oui,Non,Non,1888.00,157.33,usera4da30@example.com,OJ640131
1,21,Jeune,Femme,Urbain,Est,Célibataire,Supérieur,0.00,Employés,Privé,Non,Non,Non,Non,21558.00,1796.50,userf8c8c4@example.com,AI896231
2,50,Senior,Femme,Rural,Centre,Marié,Fondamental,20.00,Ouvriers,Privé,Non,Non,Non,Non,385.00,32.08,userca7b0a@example.com,LS406476
3,61,Âgé,Femme,Urbain,Ouest,Veuf,Secondaire,35.00,Professions intermédiaires,Public,Non,Oui,Non,Non,26976.00,2248.00,user854bbd@example.com,FV882874
4,80,Adulte,Homme,Rural,Ouest,Marié,Fondamental,20.00,Ouvriers,Informel,Non,Non,Non,Non,2835.00,236.25,userdd87ea@example.com,SD753920
5,45,Senior,Homme,Urbain,Centre,Marié,Secondaire,10.00,Employés,Public,Non,Non,Non,Oui,38429.00,3202.42,user8f96fc@example.com,FF979739
6,45,Senior,Homme,Urbain,Centre,Marié,Fondamental,18.00,Ouvriers,Privé,Non,Non,Non,Non,26881.00,2240.08,userdeeea0@example.com,IG783372
7,49,Senior,Femme,Rural,Est,Marié,Secondaire,28.00,Professions intermédiaires,Privé,Oui,Non,Non,Non,307.00,25.58,usere742ec@example.com,PX722911
8,40,Adulte,Homme,Rural,Est,Marié,Secondaire,7.00,Employés,Privé,Oui,Non,Non,Non,4515.00,376.25,userdb8c5a@example.com,YU469329
9,25,Adulte,Femme,Urbain,Est,Célibataire,Secondaire,0.00,Ouvriers,Informel,Non,Oui,Non,Non,8417.00,701.42,user5f2b94@example.com,MF380910



Affichage des 5 dernières lignes du dataset :


,Age,Categorie_age,Sexe,Milieu,Region_geographique,Etat_matrimonial,Niveau_education,Annees_experience,CSP,Secteur_emploi,Propriete_immobiliere,Vehicule_motorise,Terrain_agricole,Revenu_secondaire,Revenu_Annuel,Revenu_Mensuel,Adresse_Email,CIN
39995,55,Senior,Homme,Rural,Centre,Marié,Sans niveau,34.00,Ouvriers,Privé,Non,Oui,Non,Non,268.00,22.33,user9bb888@example.com,QS449293
39996,35,Adulte,Homme,Rural,Ouest,Marié,Supérieur,12.00,Cadres supérieurs,Public,Oui,Non,Non,Non,12083.00,1006.92,user81f7fd@example.com,AE177112
39997,23,Jeune,Femme,Urbain,Nord,Célibataire,Secondaire,2.00,Ouvriers,Informel,Non,Non,Non,Non,27045.00,2253.75,user144934@example.com,XL240775
39998,43,Adulte,Femme,Urbain,Est,Divorcé,Supérieur,10.00,Professions intermédiaires,Public,Oui,Non,Non,Non,27462.00,2288.50,user9eafdd@example.com,RB942896
39999,25,Adulte,Femme,Urbain,Centre,Marié,Fondamental,7.00,Ouvriers,NaN,Non,Non,NaN,Non,14429.00,1202.42,userec4223@example.com,GT505679


**Observation de l'aperçu initial :**
*   Les colonnes correspondent aux caractéristiques définies dans le script de génération, incluant les informations sociodémographiques, les biens, l'emploi et la variable cible `Revenu_Annuel`.
*   Les types de données semblent variés (numériques pour l'âge, le revenu ; textuels/catégoriels pour le sexe, le milieu, etc.).
*   Les colonnes non pertinentes (`Adresse_Email`, `CIN`) et la colonne redondante (`Revenu_Mensuel`) sont présentes comme attendu.

### 1.3. Description Générale et Statistique des Données

Nous examinons maintenant plus en détail les caractéristiques de notre dataset :
*   **Volume et Dimensions :** Nombre total d'instances (lignes) et d'attributs (colonnes).
*   **Types de Données et Valeurs Manquantes :** Nature de chaque attribut et détection des valeurs manquantes.
*   **Statistiques Descriptives :** Indicateurs clés pour les variables numériques et fréquences pour les variables catégorielles.

In [16]:
if df is not None:
    # Volume et Dimensions (déjà affiché au chargement, mais on peut le reconfirmer)
    print("--- Volume et Dimensions ---")
    num_instances, num_attributes = df.shape
    print(f"Nombre total d'instances (lignes) : {num_instances}")
    print(f"Nombre total d'attributs (colonnes) : {num_attributes}")

    # Types de Données, Valeurs Manquantes
    print("\n--- Informations sur les Attributs (Types, Valeurs Manquantes) ---")
    df.info()
else:
    print("Le DataFrame n'a pas été chargé.")

--- Volume et Dimensions ---
Nombre total d'instances (lignes) : 40000
Nombre total d'attributs (colonnes) : 18

--- Informations sur les Attributs (Types, Valeurs Manquantes) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Age                    40000 non-null  int64  
 1   Categorie_age          40000 non-null  object 
 2   Sexe                   40000 non-null  object 
 3   Milieu                 40000 non-null  object 
 4   Region_geographique    40000 non-null  object 
 5   Etat_matrimonial       39397 non-null  object 
 6   Niveau_education       40000 non-null  object 
 7   Annees_experience      39396 non-null  float64
 8   CSP                    40000 non-null  object 
 9   Secteur_emploi         37502 non-null  object 
 10  Propriete_immobiliere  39368 non-null  object 
 11  Vehicule_motorise      39437 n

**Analyse des Informations Générales (df.info()) :**
*   Le dataset contient 40 000 instances et 18 attributs, conformément à la génération.
*   Les types de données `int64`, `float64` et `object` sont présents, correspondant respectivement aux variables numériques entières (ex: `Age`), numériques à virgule (ex: `Revenu_Mensuel`), et catégorielles/textuelles (ex: `Sexe`, `Milieu`).
*   Les colonnes suivantes présentent des valeurs manquantes (nombre de valeurs non-nulles < 40000) : `Etat_matrimonial`, `Secteur_emploi`, `Revenu_secondaire`, `Propriete_immobiliere`, `Vehicule_motorise`, `Terrain_agricole`, et `Annees_experience`. Cela est conforme à l'introduction intentionnelle de valeurs manquantes (environ 1.5% par colonne ciblée) lors de la génération des données.

In [17]:
if df is not None:
    # Statistiques Descriptives pour les attributs numériques
    print("\n--- Statistiques Descriptives (Attributs Numériques) ---")
    display(df.describe(include=[np.number]).T) # .T pour transposer et améliorer la lisibilité
else:
    print("Le DataFrame n'a pas été chargé.")


--- Statistiques Descriptives (Attributs Numériques) ---


,count,mean,std,min,25%,50%,75%,max
Age,40000.00,40.60,13.45,15.00,29.00,41.00,52.00,80.00
Annees_experience,39396.00,10.54,9.99,0.00,2.00,8.00,16.00,47.00
Revenu_Annuel,40000.00,21931.96,38696.42,80.00,2998.75,13308.00,26997.00,608803.00
Revenu_Mensuel,40000.00,1827.66,3224.70,6.67,249.90,1109.00,2249.75,50733.58


**Analyse des Statistiques Numériques :**
*   **Age :** Varie de [min_age] à [max_age] (les valeurs aberrantes à 15 et 80 ans introduites sont visibles), avec une moyenne de [moy_age].
*   **Annees_experience :** La moyenne est de [moy_exp], avec un maximum plausible par rapport à l'âge. Les valeurs manquantes (NaN) sont exclues du calcul `count`.
*   **Revenu_Annuel :**
    *   La moyenne est de [valeur, ex: ~21953.9 DH], ce qui est très proche de la cible globale de 21949 DH.
    *   L'écart-type (`std`) est élevé ([valeur]), indiquant une forte dispersion des revenus.
    *   Le `min` est de [valeur, ex: ~100 DH] et le `max` de [valeur, ex: ~400000 DH], reflétant les valeurs aberrantes basses et hautes introduites.
    *   La médiane (`50%`) est de [valeur]. Si elle est significativement inférieure à la moyenne, cela confirme l'asymétrie positive (distribution étirée vers les hauts revenus).
*   **Revenu_Mensuel :** Les statistiques sont cohérentes avec `Revenu_Annuel` (divisées par 12).

In [18]:
if df is not None:
    # Statistiques Descriptives pour les attributs catégoriels (type 'object')
    print("\n--- Statistiques Descriptives (Attributs Catégoriels) ---")
    display(df.describe(include=['object']).T)
else:
    print("Le DataFrame n'a pas été chargé.")


--- Statistiques Descriptives (Attributs Catégoriels) ---


,count,unique,top,freq
Categorie_age,40000,4,Adulte,17315
Sexe,40000,2,Homme,20743
Milieu,40000,2,Urbain,25683
Region_geographique,40000,5,Centre,11956
Etat_matrimonial,39397,4,Marié,21111
Niveau_education,40000,4,Secondaire,14004
CSP,40000,6,Ouvriers,19374
Secteur_emploi,37502,3,Privé,16695
Propriete_immobiliere,39368,2,Non,27839
Vehicule_motorise,39437,2,Non,25921


**Analyse des Statistiques Catégorielles :**
*   **Milieu :** 2 valeurs uniques. La modalité `[Urbain/Rural]` est la plus fréquente avec [freq] occurrences ([pourcentage]%), ce qui correspond à la proportion cible (`P_URBAIN` ~64.3%).
*   **Sexe :** 2 valeurs uniques, avec une légère prédominance pour `[Homme/Femme]` ([pourcentage]%).
*   **Niveau_education :** 4 valeurs uniques. `[Fondamental/Secondaire]` sont les plus fréquentes.
*   **CSP :** 6 valeurs uniques. La répartition semble diversifiée.
*   **Region_geographique, Secteur_emploi, etc. :** Le nombre de valeurs uniques (`unique`) et la modalité la plus fréquente (`top`) sont conformes à leur définition dans le script de génération.
*   `Adresse_Email` et `CIN` ont un nombre élevé de valeurs uniques (proche de 40000), confirmant leur rôle de colonnes non pertinentes pour la modélisation directe.

In [19]:
if df is not None:
    print("\n--- Nombre de Valeurs Uniques par Colonne (Triées) ---")
    valeurs_uniques = pd.DataFrame(df.nunique(), columns=['Nombre_Valeurs_Uniques'])
    valeurs_uniques['Type_Donnee'] = df.dtypes # Ajoute le type de donnée pour contexte
    display(valeurs_uniques.sort_values(by='Nombre_Valeurs_Uniques', ascending=False))
else:
    print("Le DataFrame n'a pas été chargé.")


--- Nombre de Valeurs Uniques par Colonne (Triées) ---


,Nombre_Valeurs_Uniques,Type_Donnee
CIN,39999,object
Adresse_Email,39950,object
Revenu_Mensuel,21068,float64
Revenu_Annuel,21068,float64
Age,51,int64
Annees_experience,48,float64
CSP,6,object
Region_geographique,5,object
Categorie_age,4,object
Etat_matrimonial,4,object


**Analyse des Valeurs Uniques :**
*   Les colonnes `Adresse_Email` et `CIN` présentent le plus grand nombre de valeurs uniques, ce qui est attendu car elles sont conçues pour être uniques ou quasi-uniques et non pertinentes pour la prédiction.
*   `Revenu_Annuel` et `Revenu_Mensuel` ont également un grand nombre de valeurs uniques, ce qui est normal pour des variables cibles continues.
*   Les variables catégorielles comme `Sexe` (2), `Milieu` (2), `Niveau_education` (4), `CSP` (6), `Categorie_age` (4), etc., ont un nombre limité et attendu de modalités.

### 1.4. Exploration Approfondie avec YData-Profiling

Pour une compréhension plus fine des données, nous utilisons la librairie `ydata-profiling`. Elle génère un rapport HTML interactif qui fournit une analyse détaillée de chaque variable, les interactions entre variables, les corrélations, les valeurs manquantes, les doublons, et d'autres informations pertinentes.

**Note :** L'exécution de cette cellule peut prendre quelques instants.

In [20]:
# ASSUREZ-VOUS QUE CETTE CELLULE EST EXÉCUTÉE APRÈS QUE LES PRÉCÉDENTES AIENT RÉUSSI

if 'df' in locals() and df is not None and isinstance(df, pd.DataFrame) and not df.empty:
    print("Début de la génération du rapport d'exploration des données...")
    print(f"Le DataFrame a {df.shape[0]} lignes et {df.shape[1]} colonnes.")
    
    profile_output_file = "rapport_exploration_donnees_groupe.html"
    
    try:
        print("Initialisation de ProfileReport...")
        profile = ProfileReport(
            df, 
            title="Rapport d'Exploration - Revenu Annuel des Marocains (Groupe Barhrouj, El Kaddaoui, Kihel)", 
            explorative=True
            # Les options dark_mode et orange_mode ont été supprimées car elles causaient une ValidationError
        )
        print("ProfileReport initialisé. Tentative de génération du fichier HTML...")
        
        profile.to_file(profile_output_file)
        
        print(f"SUCCESS: Rapport d'exploration sauvegardé sous : '{profile_output_file}'")
        print("Veuillez vérifier si le fichier existe dans votre répertoire et ouvrez-le dans un navigateur.")
        
    except Exception as e:
        print(f"ERREUR LORS DE LA GÉNÉRATION DU RAPPORT ProfileReport:")
        import traceback
        print(traceback.format_exc()) 
        
elif 'df' not in locals() or df is None:
    print("ERREUR CRITIQUE : Le DataFrame 'df' n'a pas été chargé ou est None. Impossible de générer le rapport.")
elif not isinstance(df, pd.DataFrame):
    print(f"ERREUR CRITIQUE : 'df' n'est pas un DataFrame Pandas. Type actuel : {type(df)}. Impossible de générer le rapport.")
elif df.empty:
    print("ERREUR CRITIQUE : Le DataFrame 'df' est vide. Impossible de générer le rapport.")
else:
    print("Condition inattendue rencontrée avant la génération du rapport.")

Début de la génération du rapport d'exploration des données...
Le DataFrame a 40000 lignes et 18 colonnes.
Initialisation de ProfileReport...
ProfileReport initialisé. Tentative de génération du fichier HTML...


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 18/18 [00:01<00:00,  9.31it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

SUCCESS: Rapport d'exploration sauvegardé sous : 'rapport_exploration_donnees_groupe.html'
Veuillez vérifier si le fichier existe dans votre répertoire et ouvrez-le dans un navigateur.


**Analyse du Rapport d'Exploration (`rapport_exploration_donnees_groupe.html`) :**

1.  **Overview Tab :**
    *   **Dataset statistics :**
        *   Nombre de variables : 18
        *   Nombre d'observations : 40000
        *   Cellules manquantes (Missing cells) : 6178 (soit 0.9% du total des cellules)
        *   Lignes dupliquées (Duplicate rows) : 0 (soit 0.0%)
        *   Taille totale en mémoire : 32.9 MiB
    *   **Variable types :**
        *   Numériques (Real number (ℝ)) : 4 (`Age`, `Annees_experience`, `Revenu_Annuel`, `Revenu_Mensuel`)
        *   Catégorielles : 12
        *   Text : 2 (`Adresse_Email`, `CIN`)

    *   **Alerts (16 au total) :**
        *   **Corrélations élevées (High correlation) :**
            *   `Age` est fortement corrélé avec `Annees_experience` et `Categorie_age`. *Attendu, car `Categorie_age` est directement dérivée de `Age`, et l'expérience augmente généralement avec l'âge.*
            *   `CSP` est fortement corrélé avec `Niveau_education` et `Terrain_agricole`. *Logique, le niveau d'éducation influence la CSP, et la possession d'un terrain agricole est typique de la CSP "Agriculteurs".*
            *   `Revenu_Annuel` est fortement corrélé avec `Revenu_Mensuel`. *Attendu (redondance parfaite).*
            *   *(Les autres alertes de corrélation sont des réciproques de celles-ci ou des confirmations de liens logiques).*
        *   **Valeurs manquantes (Missing) :**
            *   `Etat_matrimonial` : 603 manquantes (1.5%).
            *   `Annees_experience` : 604 manquantes (1.5%).
            *   `Secteur_emploi` : 2498 manquantes (6.2%). *Ce pourcentage plus élevé s'explique par le fait que les individus 'Inactifs' (CSP) n'ont pas de secteur d'emploi.*
            *   `Propriete_immobiliere` : 632 manquantes (1.6%).
            *   `Vehicule_motorise` : 563 manquantes (1.4%).
            *   `Terrain_agricole` : 611 manquantes (1.5%).
            *   `Revenu_secondaire` : 667 manquantes (1.7%).
            *   *Ces pourcentages sont conformes à la génération intentionnelle de données manquantes.*
        *   **Zéros (Zeros) :**
            *   `Annees_experience` a 5111 zéros (12.8%). *Représente les individus jeunes ou en début de carrière.*
        *   *[Le rapport HTML peut également signaler "Skewed" pour `Revenu_Annuel` et "High Cardinality" pour `Adresse_Email`/`CIN` sous l'onglet "Alerts". Il est bon de le vérifier et de l'ajouter si c'est le cas.]*

    *   **Reproduction :**
        *   Analyse démarrée le : 2025-05-09 01:27:23
        *   Analyse terminée le : 2025-05-09 01:27:29
        *   Durée : 6.13 secondes
        *   Version de ydata-profiling : v4.16.1 *(La version peut varier)*

2.  **Variables Tab (Analyse Détaillée) :**
    *   **Variables Numériques :**
        *   `Age` : 51 valeurs distinctes, min 15, max 80, moyenne 40.6 ans. La distribution (à voir sur l'histogramme du rapport HTML) semble relativement étalée avec les outliers introduits. Skewness de 0.027 (très faible asymétrie). Kurtosis de -1.12 (platykurtique, distribution plus aplatie que la normale).
        *   `Annees_experience` : 48 valeurs distinctes, min 0, max 47, moyenne 10.5 ans. 1.5% de NaN, 12.8% de zéros. Skewness de 1.018 (asymétrie positive, queue vers les valeurs élevées). Kurtosis de 0.28 (leptokurtique, distribution légèrement plus piquée que la normale).
        *   `Revenu_Annuel` : 21068 valeurs distinctes, min 80 DH, max 608803 DH, moyenne ~21932 DH. **Skewness de 7.528 (très forte asymétrie positive)** et **Kurtosis de 82.45 (extrêmement leptokurtique, pic très élevé et queues épaisses)**. Cela confirme que la distribution est fortement concentrée sur les bas revenus avec quelques revenus très élevés. L'histogramme du rapport HTML illustrera cela de manière frappante. La valeur la plus fréquente est 239 DH (6.7% des cas), ce qui est un outlier bas.
        *   `Revenu_Mensuel` : Statistiques (skewness, kurtosis) identiques à `Revenu_Annuel`, valeurs proportionnelles.
    *   **Variables Catégorielles (Distribution des principales modalités) :**
        *   `Categorie_age` : Adulte (43.3%), Senior (32.7%), Jeune (15.1%), Âgé (8.9%).
        *   `Sexe` : Homme (51.9%), Femme (48.1%).
        *   `Milieu` : Urbain (64.2%), Rural (35.8%). *Conforme à `P_URBAIN`.*
        *   `Region_geographique` : Centre (29.9%), Nord (24.7%), Est (15.3%), Ouest (15.2%), Sud (14.8%).
        *   `Etat_matrimonial` : Marié (52.8% des non-manquants), Célibataire (24.2%), Divorcé (11.0%), Veuf (10.6%).
        *   `Niveau_education` : Secondaire (35.0%), Fondamental (34.8%), Supérieur (15.2%), Sans niveau (15.0%).
        *   `CSP` : Ouvriers (48.4%), Employés (16.7%), Professions intermédiaires (13.3%), Agriculteurs (11.3%), Cadres supérieurs (5.3%), Inactifs (5.0%).
        *   `Secteur_emploi` : Privé (41.7% des non-manquants), Informel (39.7%), Public (12.3%). 6.2% de valeurs manquantes.
        *   `Propriete_immobiliere` : Non (69.6% des non-manquants), Oui (28.8%).
        *   `Vehicule_motorise` : Non (64.8% des non-manquants), Oui (33.8%).
        *   `Terrain_agricole` : Non (85.7% des non-manquants), Oui (12.8%).
        *   `Revenu_secondaire` : Non (79.0% des non-manquants), Oui (19.3%).
    *   **Variables Text :**
        *   `Adresse_Email` : 39950 valeurs distinctes (99.9%).
        *   `CIN` : 39999 valeurs distinctes (>99.9%).
        *   *Le rapport note quelques emails et CIN dupliqués (fréquence de 2 pour certains), une petite imperfection de la génération aléatoire d'identifiants.*

3.  **Correlations Tab (Table de corrélation de Pearson) :**
    *   **Corrélations avec `Revenu_Annuel` :**
        *   `CSP` : 0.190
        *   `Niveau_education` : 0.185
        *   `Vehicule_motorise` : 0.177
        *   `Propriete_immobiliere` : 0.168
        *   `Secteur_emploi` : 0.153
        *   `Annees_experience` : 0.138
        *   `Age` : 0.132
        *   `Revenu_secondaire` : 0.129
        *   `Milieu` : 0.116
        *   Les corrélations linéaires avec `Revenu_Annuel` sont globalement faibles à modérées. `Sexe` (0.043) et `Etat_matrimonial` (0.032) montrent des corrélations linéaires particulièrement faibles, ce qui ne signifie pas qu'elles n'ont pas d'influence (l'influence peut être non linéaire ou dépendre d'interactions).
    *   **Autres corrélations notables entre prédicteurs (multicollinéarité potentielle) :**
        *   `Age` vs `Categorie_age` : 0.857 (très forte, logique)
        *   `Age` vs `Annees_experience` : 0.675 (forte)
        *   `CSP` vs `Terrain_agricole` : 0.662 (forte)
        *   `CSP` vs `Niveau_education` : 0.585 (forte)
        *   `CSP` vs `Secteur_emploi` : 0.452 (modérée à forte)
        *   `CSP` vs `Propriete_immobiliere` : 0.434 (modérée à forte)
        *   `CSP` vs `Vehicule_motorise` : 0.402 (modérée)
    *   La corrélation entre `Revenu_Annuel` et `Revenu_Mensuel` est de 1.000, confirmant la redondance.

4.  **Missing Values Tab :**
    *   *(Ouvrez votre rapport HTML, allez à l'onglet "Missing values". Regardez la "Matrix" et le "Dendrogram". Le dendrogramme montre-t-il des regroupements de variables qui ont tendance à avoir des NaN ensemble ? Par exemple, les variables de possession de biens ont-elles des NaN qui se ressemblent ? Notez ici s'il y a des patterns ou si les NaN semblent distribués de manière plutôt indépendante (ce qui est souvent le cas pour une introduction aléatoire comme la vôtre).)*
    *   *Exemple de commentaire : "La matrice des valeurs manquantes ne révèle pas de pattern systématique évident entre les différentes variables, suggérant que les NaN ont été introduits de manière relativement indépendante, à l'exception notable de `Secteur_emploi` dont les NaN sont liés à la CSP 'Inactifs'. Le dendrogramme des valeurs manquantes ne montre pas de clusters forts entre les autres variables à NaN."*

5.  **Sample Tab :**
    *   Confirme l'aperçu des données.

**N.B. : Cohérence avec la Génération et les Objectifs du Projet**
*   L'exploration approfondie via `ydata-profiling` confirme que le dataset est riche et complexe, avec les caractéristiques et les problèmes de données (NaN, outliers, redondance, non-pertinence) introduits intentionnellement.
*   La **très forte asymétrie et le kurtosis élevé de `Revenu_Annuel`** sont des points cruciaux qui devront être traités lors de la préparation des données (ex: transformation logarithmique).
*   Les corrélations linéaires modérées avec la cible suggèrent que des modèles capables de capter des relations non linéaires ou des interactions entre variables pourraient être plus performants.
*   Concernant les cibles statistiques HCP pour la **répartition des revenus** (% < moyenne) :
    *   *(Ajoutez ici les résultats de la cellule de code que vous exécuterez pour recalculer ces pourcentages, comme suggéré précédemment).*
    *   *Exemple : "Après recalcul : % sous moyenne globale: 67.4% (Cible HCP: 71.8%), % sous moyenne urbaine: 65.3% (Cible HCP: 65.9%), % sous moyenne rurale: 84.3% (Cible HCP: 85.4%). Les cibles urbaine et rurale sont bien respectées. La déviation pour la répartition globale (-4.4%) est confirmée."*



## 2. Préparation des Données

Après avoir compris nos données, l'étape suivante consiste à les préparer pour la modélisation. Cela inclut le nettoyage (gestion des doublons, des valeurs manquantes, des outliers) et la transformation (encodage des variables catégorielles, normalisation, ingénierie de caractéristiques).
Conformément aux spécifications, ces étapes seront, autant que possible, intégrées dans des pipelines Scikit-Learn pour assurer un workflow robuste et reproductible.

### 2.1. Nettoyage des Données

Suite aux résultats de l'analyse exploratoire, nous allons procéder au nettoyage des données. Cela implique :
*   L'élimination des éventuels doublons.
*   Le traitement des valeurs manquantes.
*   Le traitement des valeurs aberrantes (outliers).

Nous allons commencer par créer une copie de notre DataFrame pour cette phase de préparation et séparer les caractéristiques (X) de la variable cible (y).

In [10]:
if df is not None:
    # Créer une copie pour ne pas modifier le df original chargé
    df_processed = df.copy()

    # Séparation des caractéristiques (X) et de la variable cible (y)
    # La variable cible est 'Revenu_Annuel'
    X = df_processed.drop('Revenu_Annuel', axis=1)
    y = df_processed['Revenu_Annuel']

    print("Caractéristiques X (premières lignes) :")
    display(X.head())
    print("\nVariable cible y (premières lignes) :")
    display(y.head())
    print(f"\nDimensions de X : {X.shape}")
    print(f"Dimensions de y : {y.shape}")
else:
    print("DataFrame df non chargé. Exécution interrompue.")
    # Gérer l'erreur ou arrêter le notebook

Caractéristiques X (premières lignes) :


,Age,Categorie_age,Sexe,Milieu,Region_geographique,Etat_matrimonial,Niveau_education,Annees_experience,CSP,Secteur_emploi,Propriete_immobiliere,Vehicule_motorise,Terrain_agricole,Revenu_secondaire,Revenu_Mensuel,Adresse_Email,CIN
0,43,Adulte,Homme,Rural,Nord,Célibataire,Secondaire,12.00,Employés,Privé,Oui,Oui,Non,Non,157.33,usera4da30@example.com,OJ640131
1,21,Jeune,Femme,Urbain,Est,Célibataire,Supérieur,0.00,Employés,Privé,Non,Non,Non,Non,1796.50,userf8c8c4@example.com,AI896231
2,50,Senior,Femme,Rural,Centre,Marié,Fondamental,20.00,Ouvriers,Privé,Non,Non,Non,Non,32.08,userca7b0a@example.com,LS406476
3,61,Âgé,Femme,Urbain,Ouest,Veuf,Secondaire,35.00,Professions intermédiaires,Public,Non,Oui,Non,Non,2248.00,user854bbd@example.com,FV882874
4,80,Adulte,Homme,Rural,Ouest,Marié,Fondamental,20.00,Ouvriers,Informel,Non,Non,Non,Non,236.25,userdd87ea@example.com,SD753920



Variable cible y (premières lignes) :


0    1888.00
1   21558.00
2     385.00
3   26976.00
4    2835.00
Name: Revenu_Annuel, dtype: float64


Dimensions de X : (40000, 17)
Dimensions de y : (40000,)
